In [2]:
import os
import random
import shutil

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
def dividir_dataset_y_generar_txts(ruta_base, ruta_salida, proporciones=(0.7, 0.2, 0.1), seed=42, ruta_para_txt=None):
    """
    Divide un dataset en train/val/test y genera train.txt, val.txt y test.txt para YOLOv3/v4.

    ruta_base: carpeta donde están las imágenes y etiquetas.
    ruta_salida: carpeta donde se guardarán los conjuntos divididos.
    proporciones: (train, val, test) en porcentaje.
    seed: semilla para reproducibilidad.
    ruta_para_txt: ruta que quieres escribir en los .txt (relativa o absoluta).
    """
    # Crear carpetas
    for carpeta in ['train/images', 'train/labels', 'val/images', 'val/labels', 'test/images', 'test/labels']:
        os.makedirs(os.path.join(ruta_salida, carpeta), exist_ok=True)

    # Listar imágenes
    imagenes = [f for f in os.listdir(ruta_base) if f.lower().endswith('.jpg')]

    # Mezclar aleatoriamente
    random.seed(seed)
    random.shuffle(imagenes)

    total = len(imagenes)
    train_end = int(proporciones[0] * total)
    val_end = train_end + int(proporciones[1] * total)

    conjuntos = {
        'train': imagenes[:train_end],
        'val': imagenes[train_end:val_end],
        'test': imagenes[val_end:]
    }

    rutas_txt = {'train': [], 'val': [], 'test': []}  # Ahora también test

    for conjunto, archivos in conjuntos.items():
        for img_nombre in archivos:
            img_src = os.path.join(ruta_base, img_nombre)
            img_dst = os.path.join(ruta_salida, f'{conjunto}/images', img_nombre)

            label_nombre = os.path.splitext(img_nombre)[0] + '.txt'
            label_src = os.path.join(ruta_base, label_nombre)
            label_dst = os.path.join(ruta_salida, f'{conjunto}/labels', label_nombre)

            # Copiar imagen y label
            shutil.copy(img_src, img_dst)
            if os.path.exists(label_src):
                shutil.copy(label_src, label_dst)

            # Añadir ruta al .txt
            if ruta_para_txt:
                ruta_img = os.path.join(ruta_para_txt, conjunto, 'images', img_nombre)
            else:
                ruta_img = os.path.abspath(img_dst)
            rutas_txt[conjunto].append(ruta_img)

    # Crear archivos train.txt, val.txt, test.txt
    for conjunto in ['train', 'val', 'test']:
        txt_path = os.path.join(ruta_salida, f'{conjunto}.txt')
        with open(txt_path, 'w') as f:
            for ruta in rutas_txt[conjunto]:
                f.write(ruta + '\n')

    print("✅ División y archivos train.txt / val.txt / test.txt completados.")


In [8]:
ruta_base = "/content/drive/MyDrive/modelos_tesis/Images_Labels"
ruta_salida = "/content/drive/MyDrive/modelos_tesis/dataset_dividido"
ruta_para_txt = "/content/drive/MyDrive/modelos_tesis/dataset_dividido"

In [9]:
dividir_dataset_y_generar_txts(ruta_base, ruta_salida, proporciones=(0.7, 0.2, 0.1), ruta_para_txt=ruta_para_txt)

✅ División y archivos train.txt / val.txt / test.txt completados.
